# Part 1: introducing Pandas

Those of you who have never used jupyter notebooks before, we have an interactive python session with inline documentation using markdown.

We will start by loading in our raw data using a library called pandas.

Pandas (short for panel data) gives us access to a dataframe, which is an abstraction allowing us to hold data in a more convenient object than a list or array. The dataframe is also used in languages like R and frameworks like Spark to make processing data easy. Think of it like a spreadsheet on steroids, with a huge amount of built in functionality for manipulating our data. 

Pandas is an open-sourced library. There is a huge community of data scientists and analysts who use it and contribute to it, meaning it is high quality, under continuous development, and any bugs are fixed very rapidly.

In [ ]:
# we load a library into our namespace like so
import pandas as pd

In [ ]:
# pandas can easily read in data from a number of sources
concrete_raw = pd.read_csv("raw/concrete.csv", index_col = 0)

In [ ]:
# we can easily take a peek with the head() method
concrete_raw.head()

In [ ]:
# we can also get a summary of our data with the info() method
concrete_raw.info()

pandas allows us to answer practically any question we might have about the data. The API docs can be found [here]( http://pandas.pydata.org/pandas-docs/stable/api.html).

The library is huge, so unless it is all you use every day it is hard to memorise. From experience, you can begin to  understand what type of manipulations and summaries are possible, and then you can easily consult the API docs or Stack Overflow to solve your problems.

# Lets have a little play with pandas

In [ ]:
# how to we select data?

# all rows, one column
# note a 'series' is returned here
concrete_raw.loc[:, 'CompressiveStrength'].head()

In [ ]:
# select the first three rows
concrete_raw.loc[0:3, 'CompressiveStrength'].head()

In [ ]:
# pass in a list to access multiple columns (note a dataframe is returned)
concrete_raw.loc[0:3, ['Water', 'CompressiveStrength']].head()

In [ ]:
# we can apply summary functions to columns
concrete_raw.loc[:, 'CompressiveStrength'].mean()

In [ ]:
# we can peform groupings and aggregation
# what is the average compressive strength for different ages of concrete?
(
    concrete_raw
        .groupby('Age')
        .agg({'CompressiveStrength': {'avg': 'mean', 'n_obs': 'count'}})
        .reset_index()
        .sort_values(by='Age')   
).head()

In [ ]:
# we can query by filtering rows based on conditions
concrete_raw.loc[concrete_raw['Age'] == 28, :].head()

In [ ]:
# or a different syntax
concrete_raw.query('Age == 28').head()

In [ ]:
# we can create new columns
concrete_raw.assign(cStrengthSq = lambda x: x.CompressiveStrength ** 2).head()

# Pandas is really useful!

It has a huge set of functionality. Unfortunately it can be a bit overwhelming at first, and requires a bit of a learning curve to get used to it, and there are often multiple ways to accomplish the same task.

If you deal with a lot of data, it is far more flexible and resilient than Excel or other spreadsheet programs.

Practice is by far the best way to get to grips with it, so why not try it for some of your work? You can write scripts which run end to end to take your data from raw to processed, which means you have fully reproducible work!

# Lets do some useful work on our data now

## 1) Is there any missing data? 

We determine which rows are null using [pd.isnull](http://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.isnull.html)

We  can get a series of true or false for each row, depending if they contain nulls, like this: concrete_raw.isnull().any(axis=1)

In [ ]:
# lets use this result to filter the rows in our dataframe that contain missing data
concrete_raw.loc[concrete_raw.isnull().any(axis = 1), :]

## 2) Are there any duplicates inputs? How might we deal with these?

In [ ]:
# first, lets create a list of the feature names to easily select them

outcome_name = 'CompressiveStrength'

feature_names = concrete_raw.columns.values.tolist()
feature_names.remove(outcome_name)

print(feature_names)
print(outcome_name)

In [ ]:
# the duplicated method will give a series of true or false if the row is duplicated anywhere
# we can sum this boolean matrix (true = 1, false = 0) to let us know how many rows are duplicates
(
    concrete_raw.loc[:, feature_names]
        .duplicated()
        .sum()
)

In [ ]:
# lets take a strategy to resolve this:
# where we see multiple measurements for the same mixture,
# we will average the observations

concrete = (
   concrete_raw
        .groupby(feature_names)
        .agg('mean')
        .reset_index()
)

print("old dimensions: {} rows, {} columns".format(concrete_raw.shape[0], concrete_raw.shape[1]))
print("new dimensions: {} rows, {} columns".format(concrete.shape[0], concrete.shape[1]))

# Lets wrap this up by writing out processed data to disk



In [ ]:
concrete.to_csv("processed/concrete_processed.csv", header = True, index = False)